In [33]:
import pandas as pd

# 경로 예시 (환경에 맞게 수정)
sid_path        = "./cohort/cohort_ver50_only_subject_id.csv"
patients_path   = "../../data/MIMIC4-hosp-icu/patients.csv"
admissions_path = "../../data/MIMIC4-hosp-icu/admissions.csv"
diagnoses_path  = "../../data/MIMIC4-hosp-icu/diagnoses_icd.csv"

# --------------- 1) subject_id 리스트 ---------------
sid_df = pd.read_csv(sid_path)
if "subject_id" not in sid_df.columns:
    sid_df.columns = ["subject_id"]

target_sids = sid_df["subject_id"].dropna().astype(int).unique()

# --------------- 2) patients / admissions / diagnoses 로드 ---------------
patients   = pd.read_csv(patients_path)   # subject_id, dod
admissions = pd.read_csv(admissions_path)
diagnoses  = pd.read_csv(diagnoses_path)

patients   = patients[["subject_id", "dod"]]
admissions = admissions[["subject_id", "hadm_id", "admittime", "dischtime"]]
diagnoses  = diagnoses[["subject_id", "hadm_id", "icd_code", "icd_version"]]

admissions["admittime"] = pd.to_datetime(admissions["admittime"], errors="coerce")
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"], errors="coerce")
patients["dod"]         = pd.to_datetime(patients["dod"], errors="coerce")

# 대상 subject만
admissions = admissions[admissions["subject_id"].isin(target_sids)].copy()
diagnoses  = diagnoses[diagnoses["subject_id"].isin(target_sids)].copy()

# --------------- 3) STEMI 여부 플래그 함수 ---------------
def is_stemi_icd(row):
    code = str(row["icd_code"])
    ver  = int(row["icd_version"])

    if ver == 10:
        # ICD-10-CM: STEMI ~ I21.0–I21.3, I22.0–I22.2 (필요하면 리스트 조정)
        stemi10_prefix = ("I21.0", "I21.1", "I21.2", "I21.3", "I22.0", "I22.1", "I22.2")
        return code.startswith(stemi10_prefix)

    if ver == 9:
        # ICD-9-CM: 410.x1 중 NSTEMI(410.7x)는 제외
        # 예: 41001, 41011, 41021, 41031, 41041, 41051, 41061, 41081, 41091
        code_str = code.replace(".", "")  # 혹시 . 찍혀 있으면 제거
        if not code_str.startswith("410") or len(code_str) < 5:
            return False
        fourth = code_str[3]  # x
        fifth  = code_str[4]  # y
        if fourth == "7":     # 410.7x = NSTEMI
            return False
        return fifth == "1"   # x1 = acute episode

    return False

diagnoses["is_stemi"] = diagnoses.apply(is_stemi_icd, axis=1)

stemi_diag = diagnoses[diagnoses["is_stemi"]].copy()


In [34]:
# STEMI hadm만
adm_stemi = admissions.merge(
    stemi_diag[["subject_id", "hadm_id"]].drop_duplicates(),
    on=["subject_id", "hadm_id"],
    how="inner",
)

adm_stemi = adm_stemi.dropna(subset=["admittime"]).copy()

# 각 subject_id별로 first STEMI 입원 (admittime 가장 이른 것)
adm_stemi = (
    adm_stemi.sort_values("admittime")
             .groupby("subject_id", as_index=False)
             .first()
)

# dod 붙이기
cohort = adm_stemi.merge(patients, on="subject_id", how="left")
# cohort: subject_id, hadm_id, admittime, dischtime, dod


In [35]:
# 예시 경로 (환경에 맞게 수정)
labevents_path  = "../../data/MIMIC4-hosp-icu/labevents.csv"
dlabitems_path  = "../../data/MIMIC4-hosp-icu/d_labitems.csv"

labevents  = pd.read_csv(labevents_path)
dlabitems  = pd.read_csv(dlabitems_path)

# lab 항목 이름 기반으로 troponin itemid만 뽑기 (label, category 등 활용)
troponin_items = dlabitems[
    dlabitems["label"].str.contains("TROPONIN", case=False, na=False)
]["itemid"].unique()

trop = labevents[
    labevents["itemid"].isin(troponin_items)
][["subject_id", "hadm_id", "charttime", "valuenum", "value"]].copy()

trop["charttime"] = pd.to_datetime(trop["charttime"], errors="coerce")

# cohort의 (subject_id, hadm_id)만
key_pairs = cohort[["subject_id", "hadm_id"]]
trop = trop.merge(key_pairs, on=["subject_id", "hadm_id"], how="inner")


In [36]:
# 경로 예시
procedureevents_path   = "../../data/MIMIC4-hosp-icu/procedureevents.csv"
ditems_path            = "../../data/MIMIC4-hosp-icu/d_items.csv"
procedures_icd_path    = "../../data/MIMIC4-hosp-icu/procedures_icd.csv"
d_icd_procedures_path  = "../../data/MIMIC4-hosp-icu/d_icd_procedures.csv"

procedureevents  = pd.read_csv(procedureevents_path)
d_items          = pd.read_csv(ditems_path)
procedures_icd   = pd.read_csv(procedures_icd_path)
d_icd_procedures = pd.read_csv(d_icd_procedures_path)

# ① procedureevents + d_items 기반 PCI 후보
pe = procedureevents.merge(d_items[["itemid", "label"]], on="itemid", how="left")

pci_pe = pe[
    pe["label"].str.contains("PERCUTANEOUS", case=False, na=False) |
    pe["label"].str.contains("CORONARY", case=False, na=False) |
    pe["label"].str.contains("PCI", case=False, na=False)
][["subject_id", "hadm_id", "starttime", "endtime", "label"]].copy()

pci_pe["starttime"] = pd.to_datetime(pci_pe["starttime"], errors="coerce")
pci_pe["endtime"]   = pd.to_datetime(pci_pe["endtime"], errors="coerce")

pci_pe = pci_pe.merge(key_pairs, on=["subject_id", "hadm_id"], how="inner")

# ② procedures_icd + d_icd_procedures 기반 PCI 후보 (보조)
picd = procedures_icd.merge(
    d_icd_procedures[["icd_code", "long_title"]],
    on="icd_code", how="left"
)

pci_icd = picd[
    picd["long_title"].str.contains("percutaneous", case=False, na=False) &
    picd["long_title"].str.contains("coronary", case=False, na=False)
][["subject_id", "hadm_id", "icd_code", "long_title"]].copy()

# PCI 시간정보는 보통 procedureevents 쪽이 더 직접적이므로,
# event log에는 pci_pe의 starttime/endtime을 주로 사용.
pci = pci_pe


In [42]:
import pandas as pd
import re

# 1. ECG 원본 로드
ecg_exam_path = "../../data/mimic-iv-ecg/machine_measurements.csv"
ecg_exam = pd.read_csv(ecg_exam_path)

# 2. 사용할 컬럼만 선택
#   - subject_id
#   - ecg_time (ECG 시행 시각)
#   - report_0 ~ report_17 (진단 텍스트)
report_cols = [col for col in ecg_exam.columns if col.startswith("report_")]

ecg = ecg_exam[["subject_id", "ecg_time"] + report_cols].copy()
ecg["ecg_time"] = pd.to_datetime(ecg["ecg_time"], errors="coerce")

# 3. STEMI 플래그 생성
#    - 예시: "STEMI", "ST elevation", "ST-elevation", "acute MI" 등 포함 여부로 플래그
#    - 실제로 어떤 키워드를 쓸지는 프로젝트 기준에 맞춰 조정 가능
stemi_pattern = re.compile(r"STEMI|ST elevation|ST-elevation|acute myocardial infarction",
                           flags=re.IGNORECASE)

def has_stemi_text(row):
    texts = row[report_cols].astype(str)
    return texts.apply(lambda x: bool(stemi_pattern.search(x))).any()

ecg["stemi_flag"] = ecg.apply(has_stemi_text, axis=1).astype(int)

# 4. key_pairs와 매칭
#    - key_pairs에는 최소한 다음 컬럼이 있다고 가정:
#      ["subject_id", "hadm_id", "admittime", "dischtime"]
#    - ECG 시간(ecg_time)이 입원 구간 [admittime, dischtime] 안에 들어가는 행만 남김
key_cols = ["subject_id", "hadm_id", "admittime", "dischtime"]
key_tmp = key_pairs[key_cols].copy()
key_tmp["admittime"] = pd.to_datetime(key_tmp["admittime"], errors="coerce")
key_tmp["dischtime"] = pd.to_datetime(key_tmp["dischtime"], errors="coerce")

# subject_id 기준으로 1차 조인 (시간 조건은 나중에 필터링)
ecg = ecg.merge(key_tmp, on="subject_id", how="left")

# 5. 시간으로 admission 매칭 (ecg_time이 해당 입원 구간 안에 있는 경우만)
mask = (ecg["ecg_time"] >= ecg["admittime"]) & (ecg["ecg_time"] <= ecg["dischtime"])
ecg = ecg[mask].copy()

# 6. 이후 사용할 최소 컬럼만 정리
#    - charttime 대신 ecg_time 사용
ecg = ecg[["subject_id", "hadm_id", "ecg_time", "stemi_flag"]]

# (선택) 이름을 기존 코드와 맞추고 싶다면:
ecg = ecg.rename(columns={"ecg_time": "charttime"})


C:\Users\User\AppData\Local\Temp\ipykernel_27704\965293785.py:6: DtypeWarning: Columns (16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ecg_exam = pd.read_csv(ecg_exam_path)


KeyError: "['admittime', 'dischtime'] not in index"

In [ ]:
import pandas as pd

# log: case_id, activity, timestamp, (optionally subject_id, dod, ...)

# 1. 메타 컬럼 자동 인식 (있으면 쓰고, 없으면 생략)
possible_meta_cols = ["subject_id", "dod"]
meta_cols = [c for c in possible_meta_cols if c in log.columns]

# 2. case_id별 대표 메타 정보(첫 이벤트 기준) 추출
if meta_cols:
    case_info = (
        log.sort_values("timestamp")
           .groupby("case_id", as_index=False)[meta_cols]
           .first()
    )
else:
    # 메타 컬럼이 하나도 없으면 case_id만 가진 기본 테이블 생성
    case_info = pd.DataFrame({"case_id": log["case_id"].dropna().unique()})

# 3. case_id + activity별 "첫 발생 시각"만 추출
events_first = (
    log.sort_values(["case_id", "activity", "timestamp"])
       .groupby(["case_id", "activity"], as_index=False)["timestamp"]
       .first()
)

# 4. wide pivot: activity들을 컬럼으로 펼치기
cohort_pivot = (
    events_first
    .pivot(index="case_id", columns="activity", values="timestamp")
    .reset_index()
)

# pivot 후 컬럼 인덱스 이름 제거
cohort_pivot.columns.name = None

# 5. case_info와 병합 → 최종 Petri-net cohort
cohort_petri = case_info.merge(cohort_pivot, on="case_id", how="right")

# 6. 필요하면 activity 컬럼 이름에 접미사 붙이기 (예: ED_ARRIVAL_time)
#    원하지 않으면 이 블록은 생략해도 됨
new_columns = []
for col in cohort_petri.columns:
    if col in ["case_id"] + meta_cols:
        new_columns.append(col)
    else:
        new_columns.append(f"{col}_time")
cohort_petri.columns = new_columns

# 7. 저장
cohort_petri_path = "./cohort/petri_cohort_from_log_wide.csv"
cohort_petri.to_csv(cohort_petri_path, index=False)

print("Petri-net cohort(wide) 저장 완료:", cohort_petri_path)
print(cohort_petri.head())
